<a href="https://colab.research.google.com/github/chaitanya-949/EduVisionBot/blob/main/Rag_using_Pinecone_VectorDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install \
    "pinecone[grpc]" \
    "langchain-pinecone" \
    "langchain-openai" \
    "langchain-text-splitters" \
    "langchain"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/41

In [ ]:
PINECONE_API_KEY="f62c6582-ca79-4b39-897e-6387168cbf8f"

In [ ]:
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Set the API key as an environment variable
os.environ["PINECONE_API_KEY"] = "f62c6582-ca79-4b39-897e-6387168cbf8f"

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "docs-rag-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [ ]:
print(PINECONE_API_KEY)

f62c6582-ca79-4b39-897e-6387168cbf8f


In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 24.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF

# Function to extract data from both PDF URLs and HTML URLs containing PDFs
def fetch_and_extract_data(urls):
    papers = []

    for url in urls:
        if url.endswith('.pdf'):
            # Direct PDF link
            papers.append({
                'title': 'Direct PDF Document',
                'authors': ['Unknown'],
                'pdf_url': url,
                'date': 'Unknown'
            })
        else:
            # HTML page with embedded PDF links
            response = requests.get(url)
            html_content = response.text

            # Parse HTML content using BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')

            # Extract the publication date
            date_element = soup.find('h3')
            publication_date = date_element.get_text(strip=True) if date_element else 'Unknown date'

            # Extract all titles, authors, and PDF URLs
            base_url = 'https://arxiv.org'
            entries = soup.find_all('dl')

            for entry in entries:
                titles = entry.find_all('div', class_='list-title mathjax')
                authors = entry.find_all('div', class_='list-authors')
                pdf_links = entry.find_all('a', title='Download PDF')

                for title, author, pdf_link in zip(titles, authors, pdf_links):
                    paper_title = title.get_text(strip=True).replace('Title:', '').strip()
                    paper_authors = [a.get_text(strip=True) for a in author.find_all('a')]
                    paper_pdf_url = base_url + pdf_link['href']
                    papers.append({
                        'title': paper_title,
                        'authors': paper_authors,
                        'pdf_url': paper_pdf_url,
                        'date': publication_date
                    })

    return papers

# Function to extract text from PDF
def extract_text_from_pdf(paper):
    text = f"Publication date of {paper['title']} is {paper['date']}\nTitle: {paper['title']}\nAuthors: {', '.join(paper['authors'])}\n\n"

    try:
        # Download PDF or directly process it
        response = requests.get(paper['pdf_url'], stream=True)
        document = fitz.open(stream=response.content, filetype="pdf")

        for page_num in range(len(document)):
            page = document.load_page(page_num)
            text += page.get_text()

    except Exception as e:
        text += f"Error processing PDF at {paper['pdf_url']}: {e}"

    return text

# Example usage: Provide a list of URLs (both HTML pages and direct PDF links)
urls = [
      # HTML link with embedded PDFs
    "https://arxiv.org/pdf/1706.03762.pdf"  # Direct PDF link
]

# Fetch and extract data from both HTML and direct PDF links
papers = fetch_and_extract_data(urls)

# Extract text from each PDF and print it along with title, authors, and date
for paper in papers:
    extracted_text = extract_text_from_pdf(paper)
    print(extracted_text)
    print("\n" + "="*80 + "\n")


Publication date of Direct PDF Document is Unknown
Title: Direct PDF Document
Authors: Unknown

Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
b

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document # import Document class
extracted_texts=[extracted_text]
# Split text into manageable chunks/documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=35)

documents = []
for text in extracted_texts:
    chunks = text_splitter.split_text(text)
    for chunk in chunks: # iterate through chunks and create Document objects
      documents.append(Document(page_content=chunk))

In [ ]:
from langchain_pinecone import PineconeEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
import time
# Initialize a LangChain embedding object.
model_name = "multilingual-e5-large"
embeddings = PineconeEmbeddings(
    model=model_name,
    pinecone_api_key=os.environ.get("PINECONE_API_KEY")
)

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=documents,
    index_name="docs-rag-chatbot",
    embedding=embeddings,
    namespace="wondervector5000"
)



In [ ]:
index = pc.Index(index_name)
namespace = "wondervector5000"

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0],
        namespace=namespace,
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

Streaming output truncated to the last 5000 lines.
                         0.045440674,
                         0.002161026,
                         0.039367676,
                         0.004337311,
                         0.01751709,
                         0.00034546852,
                         0.019897461,
                         0.05618286,
                         -0.012626648,
                         0.010620117,
                         0.021560669,
                         -0.03527832,
                         -0.042938232,
                         0.008621216,
                         -0.014770508,
                         -0.052215576,
                         -0.0034980774,
                         0.03616333,
                         0.017929077,
                         -0.005558014,
                         -0.0030326843,
                         -0.00038599968,
                         -0.021820068,
                         -0.020309448,
                        

In [ ]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.9 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192", api_key="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")

In [ ]:
retriever = docsearch.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# Define the system instruction for reformulating the user's question

instruction_to_system = """
Given a chat history and the latest user question
which might reference context in the chat history, formulate a standalone question
which can be understood without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""
# Create a prompt template for reformulating questions
question_maker_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", instruction_to_system),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


# Define a chain that reformulates the question if needed
question_chain = question_maker_prompt | llm #| StrOutputParser()

In [ ]:
# Define the system prompt for the question-answering assistant
qa_system_prompt =  """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
There might be multiple research papers so check every document for better answer . \
If you don't know the answer,do not hallicuniate i need concise answer . Do not generate your answer.\
{context}"""

# Create a prompt template for the question-answering task
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [ ]:
# Function to determine if question needs reformulation based on chat history
def contextualized_question(input: dict):
    if input.get("chat_history"):
        return question_chain
    else:
        return input["question"]

In [ ]:
from langchain_core.runnables import RunnablePassthrough
# Create a retriever chain to fetch relevant context for the question
retriever_chain = RunnablePassthrough.assign(
        context=contextualized_question | retriever
    )

In [ ]:
# Define the Retrieval-Augmented Generation (RAG) chain
rag_chain = (
    retriever_chain
    | qa_prompt
    | llm
)

In [ ]:
chat_history = []
question = "can you help with this attention model architecture ?explain it to my 10 year old kid? "
from langchain_core.messages import AIMessage, HumanMessage

#Invoke the RAG chain with the question and chat history, and update chat history with responses
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg])
print(ai_msg)

content='I\'d be happy to help!\n\nImagine you\'re reading a book, and you come across a sentence that says "making it more difficult". You need to understand what "it" refers to, right? Maybe it\'s something you read a few pages ago, or maybe it\'s something that happened earlier in the story.\n\nThe attention model is like a special kind of helper that can focus on the important parts of the sentence to figure out what "it" means. It\'s like having a superpower that lets you zoom in on the relevant words and ignore the rest.\n\nHere\'s how it works:\n\n1. The attention model looks at the entire sentence and says, "Hey, I need to focus on something!"\n2. It then looks at each word in the sentence and decides how important each word is to understanding what "it" means.\n3. The model gives a special weight to the words that are most important, kind of like giving them a " spotlight" to shine on.\n4. The model then uses those important words to figure out what "it" refers to.\n\nIn the e

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.3 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from langchain_core.messages import HumanMessage

# Function to handle the chat interaction
def chat_complete(message, state):
    if state is None:
        state = []
    ai_msg = rag_chain.invoke({"question": message, "chat_history": state})
    state.append({"role": "user", "content": message})
    state.append({"role": "assistant", "content": ai_msg})
    response = [(msg["content"], state[i+1]["content"]) for i, msg in enumerate(state) if msg["role"] == "user"]
    return response, state

# Define the Gradio interface
with gr.Blocks() as block:
    gr.Markdown("""<h1><center> EduVisionBot </center></h1>""")
    chatbot = gr.Chatbot()
    message = gr.Textbox(placeholder="Type your Message.........")
    state = gr.State([])
    submit = gr.Button("SEND")

    submit.click(chat_complete, inputs=[message, state], outputs=[chatbot, state])
block.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2af87c0882679a3992.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1945, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1768, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py", line 494, in postprocess
    return self._postprocess_messages_tuples(cast(TupleFormat, value))
  File "/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py", line 429, in _postprocess_messages_tuples
    self._postprocess_content(message_pair

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2af87c0882679a3992.gradio.live
